In [1]:
!pip install webdriver-manager



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
!pip install mysql-connector-python sqlalchemy pymysql  


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas requests beautifulsoup4 yfinance tqdm



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import random
import time 
import mysql.connector
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import numpy as np
from time import sleep
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException


In [4]:
## HAM 1: CRAWL CODE

API_URL = 'https://inav.ice.com/api/1/tse/iopv/table?type=etf&language=en'
HEADERS = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://inav.ice.com'}

def get_etf_codes():
    """Lấy danh sách các ETF code theo đúng thứ tự API trả về."""
    try:
        r = requests.get(API_URL, headers=HEADERS, timeout=10)
        r.raise_for_status()
        rows = r.json().get("rows", [])
        # dùng list để giữ thứ tự, và loại code rỗng (nếu có)
        return [row["code"] for row in rows if row.get("code")]
    except requests.RequestException as e:
        print(f"⚠️ Lỗi khi lấy ETF codes từ API: {e}")
        return []

codes = get_etf_codes()
df = pd.DataFrame(codes, columns=["code"])


In [8]:
## HAM 2: CRAWL DATA

# --- CẤU HÌNH TIMEOUT ---
INITIAL_WAIT_TIMEOUT = 20  # Timeout cho lần crawl ban đầu
RETRY_WAIT_TIMEOUT = 30    # Timeout cho lần retry

# --- CHUẨN BỊ MÃ ETF GỐC ---

if 'df' not in locals() or not isinstance(df, pd.DataFrame) or 'code' not in df.columns:
    raise NameError("Lỗi: DataFrame 'df' chưa được định nghĩa hoặc thiếu cột 'code'. Vui lòng load df trước khi chạy.")

codes_to_process = df['code'].astype(str).tolist()
total_codes = len(codes_to_process)
print(f"✅ Đã sử dụng DataFrame 'df' thực tế. Tổng số mã cần xử lý: {total_codes}")

# --- HÀM 1: CRAWL LẦN 1 (TIMEOUT 20s) ---
def crawl_initial(codes_list):
    FULL_DATA_LIST = []
    MISSING_CODES = []
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    
    total_codes = len(codes_list)
    print(f"\n--- Bắt đầu Crawl lần 1 ({total_codes} mã, Timeout: {INITIAL_WAIT_TIMEOUT}s) ---")
    
    for index, base_code in enumerate(codes_list):
        base_code = str(base_code)
        full_code = f"{base_code}.T"
        url = f"https://finance.yahoo.com/quote/{full_code}/holdings/"
        
        print(f"[{index + 1}/{total_codes}] Đang xử lý mã: {full_code}")
        
        try:
            driver.get(url)
            WebDriverWait(driver, INITIAL_WAIT_TIMEOUT).until(
                EC.presence_of_element_located((By.XPATH, "//h3[text()='Sector Weightings']"))
            )

            content_divs = driver.find_elements(
                By.XPATH, 
                "//div[contains(@class, 'content')]//a[contains(@data-ylk, 'elm:sector')]/ancestor::div[contains(@class, 'content')]"
            )
            
            if not content_divs:
                MISSING_CODES.append(base_code)
                continue
                
            for div in content_divs:
                try:
                    sector_weighting = div.find_element(By.XPATH, ".//a").text
                    percent = div.find_element(By.XPATH, ".//span[contains(@class, 'data')]").text
                    
                    FULL_DATA_LIST.append({
                        'code': base_code,
                        'sector_weightings': sector_weighting,
                        'percent': percent
                    })
                except NoSuchElementException:
                    continue
        
        except TimeoutException:
            MISSING_CODES.append(base_code)
        except Exception:
            MISSING_CODES.append(base_code)
            
        time.sleep(random.uniform(1, 2))
        
    driver.quit()
    df_result = pd.DataFrame(FULL_DATA_LIST)
    print(f"--- Crawl lần 1 hoàn tất. {len(df_result)} hàng được thu thập. ---")
    
    return df_result, MISSING_CODES


# --- HÀM 2: CRAWL LẦN 2 (RETRY, TIMEOUT 30s) ---
def crawl_retry(missing_codes):
    data_list_retry = []
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    
    total_codes = len(missing_codes)
    print(f"\n--- Bắt đầu Retry ({total_codes} mã, Timeout: {RETRY_WAIT_TIMEOUT}s) ---")
    
    for base_code in missing_codes:
        full_code = f"{base_code}.T"
        url = f"https://finance.yahoo.com/quote/{full_code}/holdings/"
        
        try:
            driver.get(url)
            WebDriverWait(driver, RETRY_WAIT_TIMEOUT).until(
                EC.presence_of_element_located((By.XPATH, "//h3[text()='Sector Weightings']"))
            )

            content_divs = driver.find_elements(
                By.XPATH, 
                "//div[contains(@class, 'content')]//a[contains(@data-ylk, 'elm:sector')]/ancestor::div[contains(@class, 'content')]"
            )
            
            if not content_divs:
                continue
                
            for div in content_divs:
                try:
                    sector_weighting = div.find_element(By.XPATH, ".//a").text
                    percent = div.find_element(By.XPATH, ".//span[contains(@class, 'data')]").text
                    
                    data_list_retry.append({
                        'code': base_code,
                        'sector_weightings': sector_weighting,
                        'percent': percent
                    })
                except NoSuchElementException:
                    continue
        
        except TimeoutException:
            pass
        except Exception:
            pass
            
        time.sleep(1)
        
    driver.quit()
    
    df_retry_result = pd.DataFrame(data_list_retry)
    print(f"--- Retry hoàn tất. {len(df_retry_result)} hàng bổ sung được thu thập. ---")
    
    return df_retry_result


# --- HÀM 3: HỢP NHẤT VÀ SẮP XẾP ---
def merge_and_sort(df_result, df_retry_result, df_all_codes):
    
    df_final = pd.concat([df_result, df_retry_result], ignore_index=True)

    code_order = df_all_codes['code'].astype(str).tolist()
    code_map = {code: i for i, code in enumerate(code_order)}

    df_final['order'] = df_final['code'].map(code_map)

    df_final_sorted = df_final.sort_values(by=['order']).reset_index(drop=True)

    df_final_sorted = df_final_sorted.drop(columns=['order'])
    
    print("\n--- KẾT QUẢ HỢP NHẤT VÀ SẮP XẾP CUỐI CÙNG ---")
    print(f"Tổng số hàng dữ liệu cuối cùng: {len(df_final_sorted)}")
    print(df_final_sorted.head().to_markdown(index=False))
    
    return df_final_sorted



# THỰC THI QUY TRÌNH CHÍNH


# 1. Crawl lần 1 (Timeout 20s)
df_result, missing_codes_initial = crawl_initial(codes_to_process)

# 2. Crawl lần 2 (Retry cho các mã bị sót, Timeout 30s)
df_retry_result = crawl_retry(missing_codes_initial)

# 3. Hợp nhất và sắp xếp theo thứ tự mã gốc
df_final_sector_data = merge_and_sort(df_result, df_retry_result, df)

# ====================================================================

✅ Đã sử dụng DataFrame 'df' thực tế. Tổng số mã cần xử lý: 386

--- Bắt đầu Crawl lần 1 (386 mã, Timeout: 20s) ---
[1/386] Đang xử lý mã: 1305.T
[2/386] Đang xử lý mã: 1306.T
[3/386] Đang xử lý mã: 1308.T
[4/386] Đang xử lý mã: 1309.T
[5/386] Đang xử lý mã: 1311.T
[6/386] Đang xử lý mã: 1319.T
[7/386] Đang xử lý mã: 1320.T
[8/386] Đang xử lý mã: 1321.T
[9/386] Đang xử lý mã: 1322.T
[10/386] Đang xử lý mã: 1325.T
[11/386] Đang xử lý mã: 1326.T
[12/386] Đang xử lý mã: 1328.T
[13/386] Đang xử lý mã: 1329.T
[14/386] Đang xử lý mã: 1330.T
[15/386] Đang xử lý mã: 133A.T
[16/386] Đang xử lý mã: 1343.T
[17/386] Đang xử lý mã: 1345.T
[18/386] Đang xử lý mã: 1346.T
[19/386] Đang xử lý mã: 1348.T
[20/386] Đang xử lý mã: 1349.T
[21/386] Đang xử lý mã: 1356.T
[22/386] Đang xử lý mã: 1357.T
[23/386] Đang xử lý mã: 1358.T
[24/386] Đang xử lý mã: 1360.T
[25/386] Đang xử lý mã: 1364.T
[26/386] Đang xử lý mã: 1365.T
[27/386] Đang xử lý mã: 1366.T
[28/386] Đang xử lý mã: 1367.T
[29/386] Đang xử lý mã: 13

In [19]:
## HAM 3: XU LI 

# Xóa dấu % và chuyển sang float
df_final_sector_data['percent'] = df_final_sector_data['percent'].astype(str).str.replace("%","").astype(float)

# Đổi tên cột "code" → "etf_code"
df_final_sector_data = df_final_sector_data.rename(columns={'code': 'etf_code'})
df_final_sector_data = df_final_sector_data.rename(columns={'sector_weightings': 'sector'})


In [22]:
# --- Hàm 4: UPSERT MYSQL
def upsert_to_mysql(df: pd.DataFrame, table_name: str = 'etf_sectors_train'):
    """
    Thực hiện kết nối và UPSERT (Update or Insert) dữ liệu từ DataFrame vào MySQL.
    """
    try:
        # 1. Kết nối MySQL (Sử dụng thông tin của bạn)
        cnx = mysql.connector.connect(
            user='root',
            password='Phunghainam2004!',
            host='127.0.0.1',
            database='train'
        )
        cursor = cnx.cursor()

        # 2. Định nghĩa các cột
        # Bỏ cột 'id' nếu nó là AUTO_INCREMENT. Ở đây, ta dùng tất cả cột trừ 'id' để update.
        cols_to_use = [
            'etf_code', 'sector', 'percent'
        ]
        
        # 3. Tạo cú pháp SQL cho UPSERT
        col_str = ", ".join([f"`{c}`" for c in cols_to_use])
        placeholder_str = ", ".join(["%s"] * len(cols_to_use))
        
        # Tạo chuỗi cho phần ON DUPLICATE KEY UPDATE
        # Các cột sẽ được cập nhật nếu 'etf_code' đã tồn tại
        update_str = ", ".join([f"`{c}` = VALUES(`{c}`)" for c in cols_to_use])

        sql = f"""
        INSERT INTO `{table_name}` ({col_str})
        VALUES ({placeholder_str})
        ON DUPLICATE KEY UPDATE
        {update_str}
        """

        # 4. Chuẩn bị dữ liệu và thực thi
        df_to_insert = df[cols_to_use]
        data_to_insert = [
            tuple(None if pd.isna(x) else x for x in row) 
            for row in df_to_insert.itertuples(index=False, name=None)
        ]
        
        # Sử dụng executemany để chèn nhiều dòng cùng lúc (tối ưu hơn loop)
        cursor.executemany(sql, data_to_insert)
        
        # 5. Commit và Đóng kết nối
        cnx.commit()
        print(f"✅ Đã UPSERT {cursor.rowcount} dòng vào bảng `{table_name}` thành công!")

    except mysql.connector.Error as err:
        print(f"❌ Lỗi MySQL: {err}")
    
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()
        if 'cnx' in locals() and cnx and cnx.is_connected():
            cnx.close()




df_for_upsert = df_final_sector_data
upsert_to_mysql(df_for_upsert)

✅ Đã UPSERT 0 dòng vào bảng `etf_sectors_train` thành công!
